In [2]:
import peak
import hwtypes as hw

from peak import Peak, family_closure
from hwtypes.adt import Enum, Tuple

In [2]:
def alugen(op_map, num_ports, datawidth):
    class Opcode(Enum):
        Add = Enum.Auto()
        Sub = Enum.Auto()
        Neg = Enum.Auto()

    @family_closure
    def ALU_fc(family):
        Word = family.BitVector[datawidth]
        Bit = family.Bit
        T = family.get_constructor(Tuple[Word, Word, Word])

        @family.assemble(locals(), globals())
        class ALU(Peak):
            def __call__(self, inst: Opcode, inputs: T) -> Word:
                if inst == Opcode.Add:
                    return inputs[0] + inputs[1] + inputs[2]
                elif inst == Opcode.Sub:
                    return inputs[0] - inputs[2]
                else:
                    return -inputs[1]

        return ALU

    return Opcode, ALU_fc


def add3(a, b, c):
    return a + b + c


def sub(a, b):
    return a - b


def neg(a):
    return -a


NUM_INPUTS = 3
DATA_WIDTH = 16

OP_MAP = {
    'Add': (add3, (0, 1, 2)),
    'Sub': (sub, (0, 2)),
    'Neg': (neg, (1, )),
}

Opcode, ALU_fc = alugen(OP_MAP, NUM_INPUTS, DATA_WIDTH)

assert hasattr(Opcode, 'Add')
assert hasattr(Opcode, 'Sub')
assert hasattr(Opcode, 'Neg')

DataT = Tuple[hw.BitVector[16], hw.BitVector[16], hw.BitVector[16]]

ALU = ALU_fc.Py # ALU_fc(peak.family.PyFamily())

alu = ALU()

for _ in range(256):
    a = hw.BitVector.random(16)
    b = hw.BitVector.random(16)
    c = hw.BitVector.random(16)
    inputs = DataT(a, b, c)

    assert alu(Opcode.Add, inputs) == a + b + c
    assert alu(Opcode.Sub, inputs) == a - c
    assert alu(Opcode.Neg, inputs) == -b

In [6]:
#Hwtypes impl

def alugen(op_map, num_ports, datawidth):
    assert op_map  # len(op_map) > 0
    Opcode = type(Enum)('Opcode', (Enum, ), {k: Enum.Auto() for k in op_map})

    @family_closure
    def ALU_fc(family):
        Word = family.BitVector[datawidth]
        Bit = family.Bit
        T = family.get_constructor(Tuple[(Word for _ in range(num_ports))])

        def _build_ite(op_map, inst, inputs):
            term = None
            for op_name in reversed(op_map):
                f, op_indices = op_map[op_name]
                if term is None:
                    term = f(*(inputs[idx] for idx in op_indices))
                else:
                    op_enum = getattr(Opcode, op_name)
                    term = Bit(
                        op_enum == inst
                    ).ite(f(*(inputs[idx] for idx in op_indices)), term)
            assert term is not None
            return term

        @family.assemble(locals(), globals())
        class ALU(Peak):
            def __call__(self, inst: Opcode, inputs: T) -> Word:
                return _build_ite(op_map, inst, inputs)

        return ALU

    return Opcode, ALU_fc


def add3(a, b, c):
    return a + b + c


def sub(a, b):
    return a - b


def neg(a):
    return -a


NUM_INPUTS = 3
DATA_WIDTH = 16

OP_MAP = {
    'Add': (add3, (0, 1, 2)),
    'Sub': (sub, (0, 2)),
    'Neg': (neg, (1, )),
}

Opcode, ALU_fc = alugen(OP_MAP, NUM_INPUTS, DATA_WIDTH)

assert hasattr(Opcode, 'Add')
assert hasattr(Opcode, 'Sub')
assert hasattr(Opcode, 'Neg')

DataT = Tuple[hw.BitVector[16], hw.BitVector[16], hw.BitVector[16]]

ALU = ALU_fc.Py

alu = ALU()

for _ in range(256):
    a = hw.BitVector.random(16)
    b = hw.BitVector.random(16)
    c = hw.BitVector.random(16)
    inputs = DataT(a, b, c)

    assert alu(Opcode.Add, inputs) == a + b + c
    assert alu(Opcode.Sub, inputs) == a - c
    assert alu(Opcode.Neg, inputs) == -b

In [7]:
#using ast_tools
import ast_tools
from ast_tools.passes import apply_passes, loop_unroll

In [8]:
def alugen(op_map, num_ports, datawidth):
    assert op_map  # len(op_map) > 0

    Opcode = type(Enum)('Opcode', (Enum, ), {k: Enum.Auto() for k in op_map})

    @family_closure
    def ALU_fc(family):
        Word = family.BitVector[datawidth]
        Bit = family.Bit
        T = family.get_constructor(Tuple[(Word for _ in range(num_ports))])
        op_names = list(reversed(op_map))

        def build_args(inputs, op_indices):
            return (inputs[idx] for idx in op_indices)

        @family.assemble(locals(), globals())
        class ALU(Peak):
            @apply_passes([loop_unroll()])
            def __call__(self, inst: Opcode, inputs: T) -> Word:
                res = Word(0)

                for i in ast_tools.macros.unroll(range(len(op_names))):
                    op_name = op_names[i]
                    f, op_indices = op_map[op_name]
                    op_enum = getattr(Opcode, op_name)
                    if op_enum == inst:
                        res = f(*build_args(inputs, op_indices))
                return res

        return ALU

    return Opcode, ALU_fc


def add3(a, b, c):
    return a + b + c


def sub(a, b):
    return a - b


def neg(a):
    return -a


NUM_INPUTS = 3
DATA_WIDTH = 16

OP_MAP = {
    'Add': (add3, (0, 1, 2)),
    'Sub': (sub, (0, 2)),
    'Neg': (neg, (1, )),
}

Opcode, ALU_fc = alugen(OP_MAP, NUM_INPUTS, DATA_WIDTH)

assert hasattr(Opcode, 'Add')
assert hasattr(Opcode, 'Sub')
assert hasattr(Opcode, 'Neg')

DataT = Tuple[hw.BitVector[16], hw.BitVector[16], hw.BitVector[16]]

ALU = ALU_fc.Py
alu = ALU()

for _ in range(256):
    a = hw.BitVector.random(16)
    b = hw.BitVector.random(16)
    c = hw.BitVector.random(16)
    inputs = DataT(a, b, c)

    assert alu(Opcode.Add, inputs) == a + b + c
    assert alu(Opcode.Sub, inputs) == a - c
    assert alu(Opcode.Neg, inputs) == -b

In [13]:
def alugen(op_map, num_ports, datawidth):
    assert op_map  # len(op_map) > 0
    Opcode = type(Enum)('Opcode', (Enum, ), {k: Enum.Auto() for k in op_map})

    @family_closure
    def ALU_fc(family):
        Word = family.BitVector[datawidth]
        Bit = family.Bit
        T = family.get_constructor(Tuple[(Word for _ in range(num_ports))])
        op_names = list(reversed(op_map))

        def build_args(inputs, op_indices):
            return (inputs[idx] for idx in op_indices)

        @family.assemble(locals(), globals())
        class ALU(Peak):
            @apply_passes([loop_unroll()])
            def __call__(self, inst: Opcode, inputs: T) -> Word:
                op_name = op_names[0]
                f, op_indices = op_map[op_name]
                op_enum = getattr(Opcode, op_name)
                res = f(*build_args(inputs, op_indices))

                for i in ast_tools.macros.unroll(range(1, len(op_names))):
                    op_name = op_names[i]
                    f, op_indices = op_map[op_name]
                    op_enum = getattr(Opcode, op_name)
                    if op_enum == inst:
                        res = f(*build_args(inputs, op_indices))
                return res

        return ALU

    return Opcode, ALU_fc


def add3(a, b, c):
    return a + b + c


def sub(a, b):
    return a - b


def neg(a):
    return -a


NUM_INPUTS = 3
DATA_WIDTH = 16

OP_MAP = {
    'Add': (add3, (0, 1, 2)),
    'Sub': (sub, (0, 2)),
    'Neg': (neg, (1, )),
}

Opcode, ALU_fc = alugen(OP_MAP, NUM_INPUTS, DATA_WIDTH)

assert hasattr(Opcode, 'Add')
assert hasattr(Opcode, 'Sub')
assert hasattr(Opcode, 'Neg')

DataT = Tuple[hw.BitVector[16], hw.BitVector[16], hw.BitVector[16]]

ALU = ALU_fc.Py
alu = ALU()

for _ in range(256):
    a = hw.BitVector.random(16)
    b = hw.BitVector.random(16)
    c = hw.BitVector.random(16)
    inputs = DataT(a, b, c)

    assert alu(Opcode.Add, inputs) == a + b + c
    assert alu(Opcode.Sub, inputs) == a - c
    assert alu(Opcode.Neg, inputs) == -b

TypeError: __main__.neg() argument after * must be an iterable, not NoneType

In [14]:
import pysmt
from pysmt import shortcuts as sc

sf = peak.family.SMTFamily()

DataT = Tuple[sf.BitVector[16], sf.BitVector[16], sf.BitVector[16]]
asm_add = sf.get_adt_t(Opcode)(Opcode.Add)

ALU = ALU_fc.SMT
assert ALU is ALU_fc(sf)
alu = ALU()

a = sf.BitVector[16]()
b = sf.BitVector[16]()
c = sf.BitVector[16]()

res = alu(asm_add, DataT(a, b, c))

with sc.Solver('z3') as s:
    s.add_assertion((res != a + b + c).value)
    if s.solve():
        print('res != a + b + c')
        print(f'a = {s.get_value(a.value)}')
        print(f'b = {s.get_value(b.value)}')
        print(f'c = {s.get_value(c.value)}')
    else:
        print('verified')

SyntaxError: Cannot prove name `idx` is defined (<string>)

In [12]:
from ast_tools.passes import if_inline

def alugen(op_map, num_ports, datawidth, use_valid):
    assert op_map  # len(op_map) > 0
    Opcode = type(Enum)('Opcode', (Enum, ), {k: Enum.Auto() for k in op_map})

    @family_closure
    def ALU_fc(family):
        Word = family.BitVector[datawidth]
        Bit = family.Bit
        ts = [Word for _ in range(num_ports)]
        if use_valid:
            ts.append(Bit)

        if use_valid:
            out_t = Word, Bit
        else:
            out_t = Word

        T = family.get_constructor(Tuple[ts])
        op_names = list(reversed(op_map))

        def build_args(inputs, op_indices):
            return (inputs[idx] for idx in op_indices)

        @family.assemble(locals(), globals())
        class ALU(Peak):
            @apply_passes([loop_unroll(), if_inline()])
            def __call__(self, inst: Opcode, inputs: T) -> out_t:
                op_name = op_names[0]
                f, op_indices = op_map[op_name]
                op_enum = getattr(Opcode, op_name)
                res = f(*build_args(inputs, op_indices))

                for i in ast_tools.macros.unroll(range(1, len(op_names))):
                    op_name = op_names[i]
                    f, op_indices = op_map[op_name]
                    op_enum = getattr(Opcode, op_name)
                    if op_enum == inst:
                        res = f(*build_args(inputs, op_indices))
                        
                if ast_tools.macros.inline(use_valid):
                    if ~inputs[-1]:
                        res = Word(0)
                    valid_out = inputs[-1] # valid_in
                    return res, valid_out
                else:
                    return res

        return ALU

    return Opcode, ALU_fc


def add3(a, b, c):
    return a + b + c


def sub(a, b):
    return a - b


def neg(a):
    return -a


NUM_INPUTS = 3
DATA_WIDTH = 16

OP_MAP = {
    'Add': (add3, (0, 1, 2)),
    'Sub': (sub, (0, 2)),
    'Neg': (neg, (1, )),
}

USE_VALID = True

Opcode, ALU_fc = alugen(OP_MAP, NUM_INPUTS, DATA_WIDTH, USE_VALID)

assert hasattr(Opcode, 'Add')
assert hasattr(Opcode, 'Sub')
assert hasattr(Opcode, 'Neg')

DataT = Tuple[hw.BitVector[16], hw.BitVector[16], hw.BitVector[16], hw.Bit]

ALU = ALU_fc.Py
alu = ALU()

for _ in range(256):
    a = hw.BitVector.random(16)
    b = hw.BitVector.random(16)
    c = hw.BitVector.random(16)
    valid_in = hw.BitVector.random(1)[0]
    inputs = DataT(a, b, c, valid_in)
    if valid_in:
        assert alu(Opcode.Add, inputs) == (a + b + c, hw.Bit(1))
        assert alu(Opcode.Sub, inputs) == (a - c, hw.Bit(1))
        assert alu(Opcode.Neg, inputs) == (-b, hw.Bit(1))
    else:
        assert alu(Opcode.Add, inputs) == (hw.BitVector[16](0), hw.Bit(0))
        assert alu(Opcode.Sub, inputs) == (hw.BitVector[16](0), hw.Bit(0))
        assert alu(Opcode.Neg, inputs) == (hw.BitVector[16](0), hw.Bit(0))